In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.init as weight_init

from sklearn.model_selection import train_test_split

In [2]:
MODEL_DIR = "Model"
MODEL_VER = "2"
MODEL_FILE = "model.pt"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_VER)

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

# Model

In [3]:
lr = 0.01
dropout = 0.1
epochs = 200
activation = "sigmoid"

In [4]:
"""Reference: https://medium.com/@haoyunlai/pytorch-implementation-of-autoencoder-based-recommender-system-9aff6c3d1b02"""

class Encoder(nn.Module):
    """
    Class for Symmetrical AutoEncoder Network
    :param L: List of int, contains sizes of encoding layers and starts with feature size
    For example: [500, 20, 10] will result in:
      - encoder 2 layers: 500x20 and 20x10. Representation layer (z) will be 10
      - decoder 2 layers: 10x20 and 20x500, output size is 500, reconstructed.
    :param activation_fn: (default 'sigmoid') Type of activation function
    :param drop_prob: (default: 0.0) Dropout probability
    """
    def __init__(self, L, activation_fn='sigmoid', drop_prob=0.0):
        super(Encoder, self).__init__()
        layers = self.create_nn_structure(L)
        self.num_layers = len(L)
        # create controller for activation function
        self.activation_fn_nm = activation_fn
        # create dropout module
        self._drop_prob = drop_prob
        if drop_prob > 0.0:
            self.dropout = nn.Dropout(drop_prob)
        #initialize with empty list to store layers
        self.linears = nn.ModuleList([])
        self.linears.extend([nn.Linear(i[0], i[1]) for i in layers])
        
    def get_activation_fn(self):
        # user selected activation function at layers except for last layer
        if self.activation_fn_nm == 'relu':
            return nn.ReLU()
        elif self.activation_fn_nm == 'lrelu':
            return nn.LeakyReLU()
        elif self.activation_fn_nm == 'sigmoid':
            return nn.Sigmoid()
        else:
            raise ValueError('Activation function type not defined')
    
    def forward(self, x):
        for i,layer in enumerate(self.linears):
            if i <= self.num_layers-1:
                # create instance of activation function
                act_fn = self.get_activation_fn()
                # pass in the input
                x = act_fn(self.linears[i](x))
                if self._drop_prob > 0.0 and i <= int(self.num_layers/2): 
                    # apply dropout only on encode layer by control of i
                    x = self.dropout(x)
        # No activation on the last decoding layer
        x = self.linears[-1](x)
        return x

    def create_nn_structure(self, L):
        max_ind = len(L)-1
        layers = []
        for i,v in enumerate(L):
            if i < max_ind:
                #still have i+1 available, create layer tuple
                layer = [v,L[i+1]]
                layers.append(layer)
        #then inverse the layers for decoder size
        encoder_layers = layers[:]
        for l in encoder_layers[::-1]:
            decoder_layer = l[::-1]
            layers.append(decoder_layer)
        return layers

### Data Preprocessing

In [5]:
def create_index_mapping(L):
    '''
    return reindexed dict on user and items
    encoded indices starts from 1
    input: 
    * L: list of str
    outputs:
    * ind_2_item,item_2_ind: tuple of dictionary
    '''
    L = set(L)
    ind_2_item = {}
    
    for i,v in enumerate(L):
        #index start from 1
        ind_2_item[i+1] = v
    #invert the map
    item_2_ind = {v: k for k, v in ind_2_item.items()}
    return ind_2_item,item_2_ind

def reindexer(ratings_df,user_col,item_col,rating_col):
    '''
    inputs:
    * ratings_df: pandas df containing ratings/affinity for user-item pairs
    * user_col: actual col name for users
    * item_col: actual col name for items
    * rating_col: actual col name for ratings
    output:
    * ratings_df: reindexed user and item column, pandas df
    '''
    users_list = ratings_df[user_col].tolist()
    item_list = ratings_df[item_col].tolist()
    
    ind_2_user,user_2_ind = create_index_mapping(users_list)
    ind_2_item,item_2_ind = create_index_mapping(item_list)
    
    #rename ratings df
    ratings_df = ratings_df.rename(columns={user_col:'user_col',
                                            item_col:'item_col',
                                            rating_col:'rating_col'})
    
    #encode df using the 2 mappings
    ratings_df['encoded_users'] = ratings_df['user_col'].apply(lambda x:user_2_ind[x])
    ratings_df['encoded_items'] = ratings_df['item_col'].apply(lambda x:item_2_ind[x])
    
    return ratings_df[['encoded_users','encoded_items','rating_col']], ind_2_user,user_2_ind, ind_2_item,item_2_ind

def convert(data):
    new_data = []
    for id_users in range(nb_users+1):
        # each user's watched movies
        # data[:,0], first column, all rows column users
        id_items = data[:,1][data[:,0] == id_users]
        # each user's rating for that item
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        # the positions of these items are filled with ratings, creating the matrix
        ratings[id_items-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [6]:
def reindex_movies(movies, item_2_ind):
    movies['movieid'] = movies['movieid'].apply(lambda x:item_2_ind[x] if x in item_2_ind else -1)

def extract_genre_values(movies):
    genre = movies['genre']
    unique_genre = genre.unique()
    genre_values = []
    for movie_genre in unique_genre:
        mg = movie_genre.split("|")
        for g in mg:
            if g not in genre_values:
                genre_values.append(g)

    return sorted(genre_values, key=str.lower)

# get genre vector
def get_genre_vector(genre_row_val):
    mg = genre_row_val.split("|")
    gen_vec = np.zeros(len(genre_values))
    gen_index = 0
    for g in genre_values:
        if g in mg:
            gen_vec[gen_index] = 1
        gen_index += 1
    return gen_vec

# Add Genre Vector to movies dataframe
def add_movies_genre(movies):
#     movie_col = []

#     movie_2_genre = {}
#     for row in movies.iterrows():
#         print(row)
#         gen_vec = get_genre_vector(row['genre'])
#         movie_col.append(gen_vec)
#         movie_2_genre[row['movieid']] = np.array(gen_vec)

    movies['genre_vector'] = movies['genre'].apply(lambda x:np.array(get_genre_vector(x)))
    
    movie_2_genre = pd.Series(movies.genre_vector.values,index = movies.movieid).to_dict()

    return movie_2_genre


# def addgenrevector(data, movie_2_genre):
#     genre_array = []
#     movie_id_list = data['encoded_items'].tolist()
#     for movie_id in movie_id_list:
#         genre_array.append(movie_2_genre[movie_id])
#     data['genre_vector'] = genre_array
    
def get_user_genre(movies, ratings):
    genres = np.zeros(nb_genres)
    for movie, rating in zip(movies, ratings):
        if rating>3:
            genres += movie_2_genre[movie]
    return genres
    
    
def convert_with_side_info(data):
    new_data = []
    for id_users in range(nb_users+1):
        # each user's watched movies
        # data[:,0], first column, all rows column users
        id_items = data[:,1][data[:,0] == id_users]
        # each user's rating for that item
        id_ratings = data[:,2][data[:,0] == id_users]
        
        ratings = np.zeros(nb_movies)
        
        genres = get_user_genre(id_items, id_ratings)
                
        # the positions of these items are filled with ratings, creating the matrix
        ratings[id_items-1] = id_ratings
        
        new_data.append(list(np.append(ratings,genres)))
    return new_data

# Data loading and preprocessing

In [7]:
ratings = pd.read_csv('MovieLens1M/ratings.dat', delimiter = '::',header=None, engine='python')
movies = pd.read_csv('MovieLens1M/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('MovieLens1M/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

ratings.columns = ['userid','movieid','rating','timestamp']
ratings_reindex, ind_2_user,user_2_ind , ind_2_item,item_2_ind = reindexer(ratings,'userid','movieid','rating')

# Adding Side Information
movies.columns = ['movieid', 'movie', 'genre']
users.columns = ['userid', 'gender', 'age', 'occupation', 'zipcode']

users['female_user'] = (users['gender'] == 'F').astype(int)
users['male_user'] = (users['gender'] == 'M').astype(int)

reindex_movies(movies, item_2_ind)
genre_values = extract_genre_values(movies)
movie_2_genre = add_movies_genre(movies)

train, test = train_test_split(ratings_reindex,
                               stratify=ratings_reindex['encoded_users'],
                               test_size=0.1,
                               random_state=42)

training_set = np.array(train, dtype='int')
test_set = np.array(test, dtype='int')

nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_genres = len(genre_values)
  
training_set = convert_with_side_info(training_set)
test_set = convert_with_side_info(test_set)

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [8]:
print(test_set)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 2., 0.],
        [0., 0., 0.,  ..., 3., 1., 0.]])


# Training and Validation

In [11]:
autoencoder_network = Encoder([(nb_movies+nb_genres), 20,10], activation, dropout)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(autoencoder_network.parameters(), lr = lr, weight_decay = 0.5)

validationLoss = float("inf")
nb_epoch = epochs
mask = torch.Tensor([1]*nb_movies+[0]*nb_genres)
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        #Select only rating related columns to compute loss
        target_ratings = target[:, :nb_movies]
        if torch.sum(target.data > 0) > 0:
            output = autoencoder_network(input)
            output_ratings = output[:, :nb_movies]
            target.require_grad = False
            
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    
    # Validation
    test_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = Variable(test_set[id_user]).unsqueeze(0)
        target_ratings = target[:, :nb_movies]
        if torch.sum(target.data > 0) > 0:
            with torch.no_grad():
                output = autoencoder_network(input)
                output_ratings = output[:, :nb_movies]
                target.require_grad = False
                output[target == 0] = 0
                output = output*mask
                target = target*mask
                loss = criterion(output, target)
                mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
                test_loss += np.sqrt(loss.data*mean_corrector)
                s += 1.
    print(f"epoch: {epoch} train loss: {train_loss/s}|| test loss: {test_loss/s}")
    
    # Saving Model
    if validationLoss > (test_loss/s):
        validationLoss = (test_loss/s)
        torch.save(autoencoder_network.state_dict(), os.path.join(MODEL_PATH,MODEL_FILE))

epoch: 1 train loss: 1.2772270441055298|| test loss: 0.9871353507041931
epoch: 2 train loss: 0.9965866208076477|| test loss: 0.9620037078857422
epoch: 3 train loss: 0.9830197095870972|| test loss: 0.9564685225486755
epoch: 4 train loss: 0.9778309464454651|| test loss: 0.9532893896102905
epoch: 5 train loss: 0.9760851263999939|| test loss: 0.9529081583023071
epoch: 6 train loss: 0.9742476344108582|| test loss: 0.9513510465621948
epoch: 7 train loss: 0.9733386039733887|| test loss: 0.9491996765136719
epoch: 8 train loss: 0.9727285504341125|| test loss: 0.9500186443328857
epoch: 9 train loss: 0.9729252457618713|| test loss: 0.9515034556388855
epoch: 10 train loss: 0.9723675847053528|| test loss: 0.9493308663368225
epoch: 11 train loss: 0.9718551635742188|| test loss: 0.9492770433425903
epoch: 12 train loss: 0.9715361595153809|| test loss: 0.9488109946250916
epoch: 13 train loss: 0.9718716144561768|| test loss: 0.9506832957267761
epoch: 14 train loss: 0.9717561602592468|| test loss: 0.9494

epoch: 114 train loss: 0.9651744365692139|| test loss: 0.9439873099327087
epoch: 115 train loss: 0.9640746712684631|| test loss: 0.9450961947441101
epoch: 116 train loss: 0.9641889333724976|| test loss: 0.9433709383010864
epoch: 117 train loss: 0.9651590585708618|| test loss: 0.9442311525344849
epoch: 118 train loss: 0.9645746946334839|| test loss: 0.9436496496200562
epoch: 119 train loss: 0.9653933048248291|| test loss: 0.9464319944381714
epoch: 120 train loss: 0.966421365737915|| test loss: 0.9461266994476318
epoch: 121 train loss: 0.9665314555168152|| test loss: 0.9466121792793274
epoch: 122 train loss: 0.9664242267608643|| test loss: 0.9462850093841553
epoch: 123 train loss: 0.9665544033050537|| test loss: 0.946110725402832
epoch: 124 train loss: 0.9662100076675415|| test loss: 0.9462962746620178
epoch: 125 train loss: 0.9664932489395142|| test loss: 0.94661545753479
epoch: 126 train loss: 0.9662473201751709|| test loss: 0.9461702704429626
epoch: 127 train loss: 0.9663709998130798|

In [ ]:
# autoencoder_network = Encoder([nb_movies, 20,10], activation, dropout)
# criterion = nn.MSELoss()
# optimizer = optim.RMSprop(autoencoder_network.parameters(), lr = lr, weight_decay = 0.5)

# validationLoss = float("inf")
# nb_epoch = epochs

# for epoch in range(1, nb_epoch + 1):
#     train_loss = 0
#     s = 0.
#     # s is the number of users who rated at least 1 movies
#     for id_user in range(nb_users):
#         input = Variable(training_set[id_user]).unsqueeze(0)
#         target = input.clone()
#         if torch.sum(target.data > 0) > 0:
#             output = autoencoder_network(input)
#             target.require_grad = False
#             output[target == 0] = 0
#             loss = criterion(output, target)
#             mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) #making this anyway not equal to 0, as this will be a denominator
#             #mean_corrector is the avg of the error, only considering the movies having ratings (non-zero ratings) for computing mean of error
#             loss.backward() # decide the direction the increment of weights
#             #this call will just computing all the gradients required
#             train_loss += np.sqrt(loss.data*mean_corrector)
#             s += 1.
#             optimizer.step() # decide the amount to update the weights
    
#     # Validation
#     test_loss = 0
#     s = 0.

#     res = []
#     targets = []

#     # averaged difference between real rating and predicted rating
#     for id_user in range(nb_users):
#         input = Variable(training_set[id_user]).unsqueeze(0) # should keep the training set
#         target = Variable(test_set[id_user]).unsqueeze(0) # to predict the other movies user not seen yet

#         if torch.sum(target.data > 0) > 0:
#             # make predictions
#             with torch.no_grad():
#                 output = autoencoder_network(input)
#                 targets.append(target.detach().numpy())
#                 res.append(output.detach().numpy()) 
#                 target.require_grad = False
#                 output[target == 0] = 0 # dont want to measure the loss on the movies didnt get the actual rating from user 
#                 # force to 0 and difference / loss will be 0 for those entries
#                 loss = criterion(output, target)

#                 mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) 
#                 # only consider the movies that are rated in the test set, to be included in the loss
#                 test_loss += np.sqrt(loss.data*mean_corrector)
#                 s += 1.
#     print(f"epoch: {epoch} train loss: {train_loss/s}|| test loss: {test_loss/s}")
    
#     # Saving Model
#     if validationLoss > (test_loss/s):
#         validationLoss = (test_loss/s)
#         torch.save(autoencoder_network.state_dict(), os.path.join(MODEL_PATH,MODEL_FILE))
        
    
    


In [12]:
print(validationLoss)

tensor(0.9416)


# Making top k recommendation

In [ ]:
def make_top_k_recommendations(encoder,evidence,k,filter_seen=True):
    '''
    :param encoder: autoencoder instance
    :param evidence: full set of seen ratings from all users
    :param k: top k items (by output score)
    :param filter_seen: (default True) filter controller to remove seen items from top k list
    '''     
    res = []
    nb_users = evidence.shape[0]
    # to find top scored items for each user
    for id_user in range(nb_users):
        encoder_input = Variable(evidence[id_user]).unsqueeze(0) # should keep the training set 
        encoder_output = encoder(encoder_input)
        
        target = Variable(evidence[id_user]).unsqueeze(0) # mask to find items not seen yet
        if filter_seen:
            encoder_output[target != 0] = 0 # force seen items scores to 0, will never get recommended
        res.append(encoder_output.detach().numpy())
        
    res = [a[0] for a in res]
    final_itemsets = []    
    for each in res:
        full_ratings_predicted = list(each)
        full_ratings_indexed = list(enumerate(full_ratings_predicted))
        final_itemsets.append(sorted(full_ratings_indexed,key=lambda x:x[1],reverse =True)[:k])
        
    return final_itemsets